In [1]:
%run drop_test_simulator.ipynb
import pandas as pd

In [2]:
heli_direction = 180 # deg on unit circle
amount = 2500 # number of simulations of each possible scenario
time_step = 0.05 # s
nominal_airspeed=15.43334 # m/s, this is 30 knots

# note that if dt=0.25 the horizontal distance may be off by about 8 m (compared to dt=0.001)
def mc_init(num_sims, dt, mode, airspeed, wind_dir):
    all_coords = []
    v_wind_rndm = []
    v_plane_rndm = []
    v_side_ref = []
    init_pos_offset = []

    deg_wind_mean, deg_wind_std_dev  = (wind_dir, 10) # 20-40-60 degree cone (with probability 68-95-99.7)
    
    for i in range(num_sims):
        v_wind_rndm.append([np.random.rayleigh(2.47344), # rayleigh distro for wind magnitude at 10 m
                            np.random.normal(deg_wind_mean, deg_wind_std_dev)])
        
        actual_airspeed = np.random.normal(airspeed, 1.715) # +- 10 knot tolerance (3 sigma)
        
        # convert airspeed to ground speed by adding components of wind velocity at release altitude
        v_plane_rndm.append(actual_airspeed -
                            wind_profile(609.6/2, v_wind_rndm[-1][0]) *
                            np.cos(np.radians(v_wind_rndm[-1][1])))
        v_side_ref.append(wind_profile(609.6/2, v_wind_rndm[-1][0]) *
                          np.sin(np.radians(v_wind_rndm[-1][1])))
        
        init_pos_offset.append([np.random.normal(0, 30 * 0.3048), # converting feet to meters
                                np.random.normal(0, 30 * 0.3048),
                                np.random.normal(0, 15 * 0.3048)]) # account for the imperfections of reality
    print('random wind and helicopter vectors obtained\n')

    for i, vw in enumerate(v_wind_rndm):
        all_coords.append(trajectory(v_plane=v_plane_rndm[i],
                                     v_side=v_side_ref[i],
                                     deg_plane=heli_direction,
                                     v_wind=vw[0],
                                     deg_wind=vw[1],
                                     x_0_offset=init_pos_offset[i],
                                     mode=mode,
                                     dt=dt)[0])
        if i % 250 == 0: print('iterations:', i)
    print('done simulations!\n')
    
    return (all_coords, v_wind_rndm, v_plane_rndm)

In [3]:
# this function assumes dataset has not been translated,
# and hence should only be run once initially to determine CEP
def polar_coord(df):
    # magnitude of landing coordinate
    df['norm'] = df.loc[:,['x','y']].apply(np.linalg.norm, axis=1, raw=True)
    # azimuth angle of landing (note that this is on the unit circle)
    df['theta'] = df.loc[:,['x','y']].apply(lambda pos: np.arctan2(pos[1], pos[0]), axis=1, raw=True)
      
def handle_data(all_coords, v_wind_rndm, v_plane_rndm, mode):
    # landing coordinates
    # sims are conducted in metric units so we convert m to ft here.
    results_df = pd.DataFrame.from_records([np.array([m_to_ft(Xi) for Xi in X])
                                            for X in all_coords],
                                           columns=['x', 'y'])
    polar_coord(results_df)
    # plane ground velocities
    results_df['v_plane'] = [ms_to_knots(vel) for vel in v_plane_rndm]
    # wind magnitude
    results_df['v_w'] = [ms_to_knots(vel[0]) for vel in v_wind_rndm]
    # wind direction as on unit circle
    results_df['deg_w'] = [vel[1] for vel in v_wind_rndm]
    results_df['mode'] = [mode for vel in v_plane_rndm]
    print(results_df.describe())
    return results_df

In [4]:
# mission success
nominal_against_coords, nominal_against_wind, nominal_against_plane = mc_init(
    num_sims=amount, dt=time_step, mode='', airspeed=nominal_airspeed, wind_dir=heli_direction-180)

# main chute failure
nomain_against_coords, nomain_against_wind, nomain_against_plane = mc_init(
    num_sims=amount, dt=time_step, mode='A', airspeed=nominal_airspeed, wind_dir=heli_direction-180)

# drogue failure
nodrogue_against_coords, nodrogue_against_wind, nodrogue_against_plane = mc_init(
    num_sims=amount, dt=time_step, mode='B', airspeed=nominal_airspeed, wind_dir=heli_direction-180)

# early deployment
early_against_coords, early_against_wind, early_against_plane = mc_init(
    num_sims=amount, dt=time_step, mode='C', airspeed=nominal_airspeed, wind_dir=heli_direction-180)

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
iterations: 1000
iterations: 1250
iterations: 1500
iterations: 1750
iterations: 2000
iterations: 2250
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
iterations: 1000
iterations: 1250
iterations: 1500
iterations: 1750
iterations: 2000
iterations: 2250
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
iterations: 1000
iterations: 1250
iterations: 1500
iterations: 1750
iterations: 2000
iterations: 2250
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
iterations: 1000
iterations: 1250
iterations: 1500
iterations: 1750
iterations: 2000
iterations: 2250
done simulations!



In [5]:
# mission success
nominal_across_coords, nominal_across_wind, nominal_across_plane = mc_init(
    num_sims=amount, dt=time_step, mode='', airspeed=nominal_airspeed, wind_dir=heli_direction-90)

# main chute failure
nomain_across_coords, nomain_across_wind, nomain_across_plane = mc_init(
    num_sims=amount, dt=time_step, mode='A', airspeed=nominal_airspeed, wind_dir=heli_direction-90)

# drogue failure
nodrogue_across_coords, nodrogue_across_wind, nodrogue_across_plane = mc_init(
    num_sims=amount, dt=time_step, mode='B', airspeed=nominal_airspeed, wind_dir=heli_direction-90)

# early deployment
early_across_coords, early_across_wind, early_across_plane = mc_init(
    num_sims=amount, dt=time_step, mode='C', airspeed=nominal_airspeed, wind_dir=heli_direction-90)

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
iterations: 1000
iterations: 1250
iterations: 1500
iterations: 1750
iterations: 2000
iterations: 2250
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
iterations: 1000
iterations: 1250
iterations: 1500
iterations: 1750
iterations: 2000
iterations: 2250
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
iterations: 1000
iterations: 1250
iterations: 1500
iterations: 1750
iterations: 2000
iterations: 2250
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
iterations: 1000
iterations: 1250
iterations: 1500
iterations: 1750
iterations: 2000
iterations: 2250
done simulations!



# Data Handling for Against Wind Simulations

In [6]:
nominal_against_data = handle_data(nominal_against_coords, nominal_against_wind, nominal_against_plane, 'nominal')
nomain_against_data = handle_data(nomain_against_coords, nomain_against_wind, nomain_against_plane, 'nomain')
nodrogue_against_data = handle_data(nodrogue_against_coords, nodrogue_against_wind, nodrogue_against_plane, 'nodrogue')
early_against_data = handle_data(early_against_coords, early_against_wind, early_against_plane, 'early')

                 x            y         norm        theta      v_plane  \
count  2500.000000  2500.000000  2500.000000  2500.000000  2500.000000   
mean    357.491197    -0.817386   385.721369     0.012868    20.385862   
std     282.970425   114.588910   268.726875     0.851744     5.950951   
min    -226.256972  -601.851207     1.606745    -3.132944    -0.160976   
25%     143.331815   -60.471456   166.002774    -0.233728    16.506740   
50%     327.014890     1.072443   343.023147     0.002962    20.660204   
75%     532.420656    57.536567   549.416944     0.228568    24.744780   
max    1332.907904   580.122364  1380.013281     3.127113    36.772514   

               v_w        deg_w  
count  2500.000000  2500.000000  
mean      6.021345    -0.188238  
std       3.136094    10.045142  
min       0.105991   -34.179258  
25%       3.617070    -6.723550  
50%       5.677847    -0.354343  
75%       7.912204     6.796703  
max      17.510152    36.854628  
                 x         

# Data Handling for Across Wind Simulations

In [7]:
nominal_across_data = handle_data(nominal_across_coords, nominal_across_wind, nominal_across_plane, 'nominal')
nomain_across_data = handle_data(nomain_across_coords, nomain_across_wind, nomain_across_plane, 'nomain')
nodrogue_across_data = handle_data(nodrogue_across_coords, nodrogue_across_wind, nodrogue_across_plane, 'nodrogue')
early_across_data = handle_data(early_across_coords, early_across_wind, early_across_plane, 'early')

                 x            y         norm        theta      v_plane  \
count  2500.000000  2500.000000  2500.000000  2500.000000  2500.000000   
mean   -196.044939   562.647179   614.165263     1.979977    30.042408   
std     115.989343   301.855983   286.998567     0.350312     3.841813   
min    -763.504831   -28.573531   106.897383    -3.113497    15.930843   
25%    -258.082147   333.292648   392.021711     1.788712    27.498989   
50%    -194.899940   524.121558   568.164491     1.953626    30.012723   
75%    -130.652163   751.093776   792.355883     2.137038    32.543856   
max     485.636967  1790.608404  1819.027648     3.139226    44.544685   

               v_w        deg_w  
count  2500.000000  2500.000000  
mean      5.930492    90.338660  
std       3.110993    10.148859  
min       0.104526    44.473779  
25%       3.544870    83.340013  
50%       5.516969    90.456456  
75%       7.940016    97.032163  
max      18.138374   123.383804  
                 x         

## Saving data and Aggregate Results

In [8]:
nominal_against_data.to_csv(path_or_buf='./drop_test_sample_data/nominal_against_data.csv')
nomain_against_data.to_csv(path_or_buf='./drop_test_sample_data/nomain_against_data.csv')
nodrogue_against_data.to_csv(path_or_buf='./drop_test_sample_data/nodrogue_against_data.csv')
early_against_data.to_csv(path_or_buf='./drop_test_sample_data/early_against_data.csv')

In [9]:
nominal_across_data.to_csv(path_or_buf='./drop_test_sample_data/nominal_across_data.csv')
nomain_across_data.to_csv(path_or_buf='./drop_test_sample_data/nomain_across_data.csv')
nodrogue_across_data.to_csv(path_or_buf='./drop_test_sample_data/nodrogue_across_data.csv')
early_across_data.to_csv(path_or_buf='./drop_test_sample_data/early_across_data.csv')